# ALENN - Replication Notebook 
## Brock and Hommes (1998) Model, MDN

Donovan Platt
<br>
Mathematical Institute, University of Oxford
<br>
Institute for New Economic Thinking at the Oxford Martin School
<br>
<br>
Copyright (c) 2020, University of Oxford. All rights reserved.
<br>
Distributed under a BSD 3-Clause licence. See the accompanying LICENCE file for further details.

# 1. Modules and Packages
Load all required modules and packages.

In [1]:
# Import the ALENN ABM Estimation Package
import alenn

# Import Numerical Computation Libraries
import numpy as np
import pandas as pd

# Import General Mathematical Libraries
from scipy import stats

# Import System Libraries
import os
import logging

Using TensorFlow backend.


In [2]:
# Disable Tensorflow Deprecation Warnings
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Tensorflow 2.x deprecates many Tensorflow 1.x methods, causing Tensorflow 1.15.0 to output a large number 
# of deprecation warnings when performing the first likelihood calculation. This can be very distracting,
# leading us to disable them.

# 2. Estimation Experiments
Replication of the MDN experiments. Note that here we generate only a single Markov Chain as opposed to the 5 considered in the original paper.

## 2.1. Free Parameter Set 1

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = alenn.models.brock_hommes(0, 0, -0.7, -0.4, 0.5, 0.3, 1.01, 0, 0.01, 10, 0.04, 10, T_emp, n, 1)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return alenn.models.brock_hommes(0, 0, theta[0], theta[1], theta[2], theta[3], 1.01, 0, 0.01, 10, 0.04, 10, T_sim, n, 7)

# Define Parameter Priors
priors = [stats.uniform(loc = -2.5, scale = 2.5).pdf,
          stats.uniform(loc = -1.5, scale = 1.5).pdf,
          stats.uniform(loc = 0, scale = 2.5).pdf,
          stats.uniform(loc = 0, scale = 1.5).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-2.5, -1.5, 0, 0])
theta_upper = np.array([0, 0, 2.5, 1.5])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 4 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 1000 observations in total.
-------------------------------------------------

### Sampler Specification

In [5]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 10000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         10000               
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                 -2.5          0.0
2                 -1.5          0.0
3                  0.0          2.5
4                  0.0          1.5
----------------------------------------------------------------------------



### Result Processing

In [14]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 5000)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1               -0.727035             0.185455
2               -0.405627             0.010080
3                0.585974             0.197566
4                0.317112             0.009391


## 2.2. Free Parameter Set 2

### Model Specification

In [6]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = alenn.models.brock_hommes(0, 0, 0.6, 0.65, 0.7, -0.55, 1.01, 0, 0.01, 10, 0.04, 10, T_emp, n, 1)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return alenn.models.brock_hommes(0, 0, theta[0], theta[1], theta[2], theta[3], 1.01, 0, 0.01, 10, 0.04, 10, T_sim, n, 7)

# Define Parameter Priors
priors = [stats.uniform(loc = 0, scale = 2.5).pdf,
          stats.uniform(loc = 0, scale = 1.5).pdf,
          stats.uniform(loc = 0, scale = 2.5).pdf,
          stats.uniform(loc = -1.5, scale = 1.5).pdf]

# Define the Parameter Bounds
theta_lower = np.array([0, 0, 0, -1.5])
theta_upper = np.array([2.5, 1.5, 2.5, 0])

### Posterior Specification

In [7]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 4 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 1000 observations in total.
-------------------------------------------------

### Sampler Specification

In [8]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 10000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         10000               
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                  0.0          2.5
2                  0.0          1.5
3                  0.0          2.5
4                 -1.5          0.0
----------------------------------------------------------------------------



### Result Processing

In [19]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 5000)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.578004             0.018663
2                0.645939             0.006415
3                0.749964             0.016154
4               -0.513151             0.004915
